In [2]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

!pip install deap

import operator
import random
import math
import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", numpy.ndarray, fitness=creator.FitnessMax, speed=list,
    smin=None, smax=None, best=None)

def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(numpy.random.uniform(pmin, pmax, size))
    part.speed = numpy.random.uniform(smin, smax, size)
    part.smin = smin
    part.smax = smax
    return part

def updateParticle(part, best, phi1, phi2):
    u1 = numpy.random.uniform(0, phi1, len(part))
    u2 = numpy.random.uniform(0, phi2, len(part))
    v_u1 = u1 * (part.best - part)
    v_u2 = u2 * (best - part)
    part.speed += v_u1 + v_u2
    for i, speed in enumerate(part.speed):
        if abs(speed) < part.smin:
            part.speed[i] = math.copysign(part.smin, speed)
        elif abs(speed) > part.smax:
            part.speed[i] = math.copysign(part.smax, speed)
    part += part.speed

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if part.best is None or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if best is None or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)

    return pop, logbook, best

if __name__ == "__main__":
    pop, logbook, best = main()

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Particle' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	evals	avg     	std      	min      	max     
0  	5    	0.142062	0.0398183	0.0801914	0.191216
1  	5    	0.125971	0.079511 	0.0482987	0.251429
2  	5    	0.166167	0.0554023	0.115664 	0.269617
3  	5    	0.206204	0.0653578	0.147397 	0.318687
4  	5    	0.281601	0.111388 	0.0943761	0.379751
5  	5    	0.367527	0.252967 	0.0708228	0.836591
6  	5    	0.29081 	0.114789 	0.196787 	0.507429
7  	5    	0.293151	0.204298 	0.0644211	0.594651
8  	5    	0.557583	0.411243 	0.228439 	1.32945 
9  	5    	0.265514	0.169779 	0.0106593	0.482583
10 	5    	0.417116	0.370381 	0.0205341	1.11217 
11 	5    	0.323967	0.0683129	0.251972 	0.454092
12 	5    	0.401347	0.365968 	0.0650187	1.10066 
13 	5    	0.299187	0.198471 	0.113366 	0.594851
14 	5    	0.21002 	0.0863872	0.0975886	0.346571
15 	5    	0.312162	0.156564 	0.0300508	0.496415
16 	5    	0.295822	0.0708891	0.204009 	0.401297
17 	5    	0.512041	0.460328 	0.151936 	1.41064 
18 	5    	0.288835	0.0684832	0.225782 	0.380357
19 	5    	0.472412	0.267105 	0.213101 	0